<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Error_Handler_Python_File%20SQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

     |████████████████████████████████| 132 kB 10.3 MB/s 
     |████████████████████████████████| 8.7 MB 64.4 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |████████████████████████████████| 138 kB 65.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import json
import boto3
import traceback
import datetime
from datetime import date

In [3]:
# Month abbreviation, day and year
today = date.today()	
d4 = today.strftime("%b-%d-%Y")

In [4]:
print(d4)

May-15-2022


In [6]:
sqs_client=boto3.client('sqs',  aws_access_key_id='',
aws_secret_access_key='', region_name='us-east-2')

s3= boto3.client('s3',aws_access_key_id='',
aws_secret_access_key='', region_name='us-east-2')

In [7]:
sqs_dlq= ""
sqs_source_queue=""
s3_prefix_stage=''
s3_bucket=""
max_retries=

In [8]:
# Defining Custom Function to Send Message to Main Queue and Deleting it from DLQ 
def sendAndDeleteQueue(sendQueueUrl,messageBody,messageAttributes,recieveQueueUrl,receiptHandle) :
    # Calling Send Message API
    ret = sqs_client.send_message( QueueUrl=sendQueueUrl, 
                            MessageBody=messageBody,
                            MessageAttributes=messageAttributes) 
                            
    # Calling Delete Message API 
    sqs_client.delete_message(QueueUrl=recieveQueueUrl, ReceiptHandle=receiptHandle)

In [9]:
def process_sqs_message(sqs_source_queue,sqs_dlq):
  while True:
    messages = sqs_client.receive_message(QueueUrl=sqs_dlq, MaxNumberOfMessages=10
                                      ,WaitTimeSeconds=20,MessageAttributeNames=['All'])
    if 'Messages' in messages:
      for m in messages['Messages']:
        new_MessageAttributes = {}                      # Declaring empty Dictonary 
        retryCount = 1                                  # Setting retry Counter as 1 since it is a New message.
        current_retry_count=0
        if 'MessageAttributes' in m :                   # Checking whether Message Attributes exists in Original Message 
          new_MessageAttributes = m['MessageAttributes']
          if 'retryCount' in m['MessageAttributes'] :
            retryCount = int(m['MessageAttributes']['retryCount']['StringValue']) + 1
            current_retry_count=int(m['MessageAttributes']['retryCount']['StringValue'])

        new_MessageAttributes['retryCount'] = {'StringValue': str(retryCount),'DataType': 'Number'} 

        if current_retry_count <= max_retries :
          sendAndDeleteQueue(sqs_source_queue,m['Body'],new_MessageAttributes,sqs_dlq,m['ReceiptHandle'])

        else:
          # Need to Store Message in S3 Bucket before finally deleting the message from recieve Queue
          ct = datetime.datetime.now().timestamp()              # Calculating Current Timestamp 
          s3_key = s3_prefix_stage+"/"+d4+"/"+str(ct)+"_"+m['MessageId']+".json"
          message_data=json.dumps({"Body":m['Body'],"Attribute":new_MessageAttributes})
          s3.put_object(Body=message_data,Bucket=s3_bucket,Key=s3_key)
          sqs_client.delete_message(QueueUrl=sqs_dlq, ReceiptHandle=m['ReceiptHandle'])  
    else:
      print('Queue is currently Empty or Messages are Invisible')
      break

In [13]:
process_sqs_message(sqs_source_queue,sqs_dlq)

Queue is currently Empty or Messages are Invisible
